In [32]:
import pandas as pd
import glob
import json
import re
import numpy as np
from IPython.display import display
set_fn = set()


In [33]:

def extract_config(line):
    line = line.rstrip(')\n')
    config = re.findall("(\w+)='?([^,']*)'?", line)                                                                                                                                                             
    return {k: v for k, v in config}             

def list_dict_reverse(L):
    """ Given a list of dict with identical keys, turn that into a dict of numpy arrays.
    """
    keys = L[0].keys()
    res = {}
    for k in keys:
        res[k] = np.asarray([l[k] for l in L])
    return res

def parse_log(fn):
    with open(fn, 'r') as f:
        cfg = extract_config(f.readline())
        train_logs, test_logs = [], []
        for line in f.readlines():
            try:
                parsed = json.loads(line)
            except json.JSONDecodeError:
                continue
            if parsed['mode'] == 'train':
                train_logs.append(parsed)
            elif parsed['mode'] == 'test':
                test_logs.append(parsed)
        train_logs = list_dict_reverse(train_logs)
        test_logs = list_dict_reverse(test_logs)
        return cfg, train_logs, test_logs
        
data = []
data_var = []
for fn in glob.glob('../res_proto_3_adam/*'):
    print("Read", fn)
    if fn in set_fn:
        print("Already parsed", fn)
        continue
    set_fn.add(fn)
    try:
        cfg, tr, te = parse_log(fn)
    except:
        continue
    cfg.update({
        'final_length': te['length'][-1],
        'final_loss': te['loss_objs'][-1],
        'final_loss_roles': te['loss_roles'][-1],

    })
    for k in te.keys():
        if k.startswith('length'):
            cfg.update({
                'init_' + k: te[k][0],
                'final_' + k: te[k][-1],
            })
    data_var.append(cfg)
df = pd.DataFrame(data_var)

Read ../res_proto_3_adam/b9edf7598b094e74bf790338c612eed1
Read ../res_proto_3_adam/67ca2f3fa5d73d76b56f1abeb0f551cf
Read ../res_proto_3_adam/733aa6f8784f6c9e9e86e90058b9abb7
Read ../res_proto_3_adam/74dfe3e70d510cbd68079e37efb7c730
Read ../res_proto_3_adam/0280f20be8a7a567772e30835d0d012c
Read ../res_proto_3_adam/19199613ab78f7d20ab918161f9f6689
Read ../res_proto_3_adam/794dbd9dac94d90d2da340da2a57ec71
Read ../res_proto_3_adam/fbdd009f20315221e41aac11fe8dac0f
Read ../res_proto_3_adam/7fdbd3550068f29b20eb1142686a8350
Read ../res_proto_3_adam/32e207da7e1a65fc538ee23354f36167
Read ../res_proto_3_adam/e096e436e51baa22dda1be5129f182d5
Read ../res_proto_3_adam/ee9bf8f40702ac7fe5dcc2a9c0b8ee0a
Read ../res_proto_3_adam/4102d78e8451d61a0e20b207df1e691b
Read ../res_proto_3_adam/9f4af41f8a10e4e418de2393d4f4df6f
Read ../res_proto_3_adam/67ca2f3fa5d73d76b56f1abeb0f551cf_I
Read ../res_proto_3_adam/63fbd0b4ddc66b48fc4393fa06f89dc2
Read ../res_proto_3_adam/1408f60b4e41d3c3c78ad40471f9423b
Read ../res_

In [34]:
df = df.apply(lambda x: pd.to_numeric(x, errors='ignore', downcast='float'))


# Factors

## Temperature
High temperature seems higher accuracy, and lower final entropy for -1...

In [36]:
# fields = ['temperature', 'optimizer', 'lr', 'mode', 'random_seed', ]
fields = ['final_loss', 'final_loss_roles','final_length', 'lr', 'random_seed', 'optimizer', 'temperature']

criterion_1 = (df['mode'] == 'gs')
# criterion_2 = (df['optimizer'] == 'adam')
#criterion_3 = (df['train_test_ratio'] == '-1.0')
print(len(df.loc[criterion_1]))
a = df.loc[criterion_1][fields].groupby(['temperature'])
print(len(a))
print(a)
a.agg(['mean', 'std', 'count'])

46
3


final_loss                  final_loss_roles                  \
                  mean        std count             mean       std count   
temperature                                                                
0.90         16.190842   6.925292    23         7.578428  0.671536    23   
1.00         19.449461   9.247333    17         7.554811  0.620354    17   
1.25         16.353355  12.215440     6         7.444898  0.523402     6   

            final_length                      lr            random_seed  \
                    mean       std count    mean  std count        mean   
temperature                                                               
0.90            5.812500  2.405102    23  0.0003  0.0    23    1.956522   
1.00            5.585503  3.076694    17  0.0003  0.0    17    1.294118   
1.25            4.651849  3.068691     6  0.0003  0.0     6    1.000000   

                             
                  std count  
temperature                  
0.90         0.975997    23  
1.00         1.104802    17  
1.25         1.549193     6

## Distance regularization

In [37]:
# fields = ['temperature', 'optimizer', 'lr', 'mode', 'random_seed', ]
fields = ['final_loss', 'final_loss_roles', 'distance_reg_coef', 'final_length', 'lr', 'random_seed', 'optimizer', 'temperature']

criterion_1 = (df['mode'] == 'gs')
# criterion_2 = (df['optimizer'] == 'adam')
#criterion_3 = (df['train_test_ratio'] == '-1.0')
print(len(df.loc[criterion_1]))
a = df.loc[criterion_1][fields].groupby(['distance_reg_coef'])
print(len(a))
print(a)
a.agg(['mean', 'std', 'count'])

46
3


final_loss                 final_loss_roles                  \
                        mean       std count             mean       std count   
distance_reg_coef                                                               
0.0                16.026466  8.989673    17         7.558182  0.442729    17   
0.1                19.563946  9.313910    10         7.550798  0.519823    10   
1.0                17.529526  7.916327    19         7.547788  0.815279    19   

                  final_length                      lr            random_seed  \
                          mean       std count    mean  std count        mean   
distance_reg_coef                                                               
0.0                   4.874410  2.577036    17  0.0003  0.0    17    1.941176   
0.1                   5.808188  3.325543    10  0.0003  0.0    10    1.600000   
1.0                   6.084488  2.501135    19  0.0003  0.0    19    1.263158   

                                  temperature                  
                        std count        mean       std count  
distance_reg_coef                                              
0.0                1.248529    17    0.994118  0.149877    17  
0.1                1.074968    10    0.980000  0.042164    10  
1.0                1.045738    19    0.973684  0.108485    19

## Optimisation

- Adam's beta_2 trades off between length and reconstruction

In [38]:

fields = ['final_loss', 'final_loss_roles', 'final_length', 'lr', 'adam_beta2', 'random_seed', 'optimizer', 'temperature', 'batch_size']
criterion_1 = (df['mode'] == 'gs')
# criterion_2 = (df['optimizer'] == 'adam')
#criterion_3 = (df['train_test_ratio'] == '-1.0')
a = df.loc[criterion_1][fields].groupby(['optimizer', 'lr', 'adam_beta2'])
display(a.agg(['mean', 'std', 'count']))


final_loss                 final_loss_roles  \
                                  mean       std count             mean   
optimizer lr     adam_beta2                                               
adam      0.0003 0.900       14.716751  8.144448    15         7.195133   
                 0.990       16.644136  7.350219    18         7.821834   
                 0.999       21.600355  9.514410    13         7.591156   

                                            final_length                  \
                                  std count         mean       std count   
optimizer lr     adam_beta2                                                
adam      0.0003 0.900       0.585717    15     6.046925  2.352347    15   
                 0.990       0.518468    18     5.581848  2.596189    18   
                 0.999       0.640302    13     5.028846  3.328423    13   

                            random_seed                 temperature            \
                                   mean       std count        mean       std   
optimizer lr     adam_beta2                                                     
adam      0.0003 0.900         1.533333  1.407463    15    1.006667  0.155686   
                 0.990         1.777778  0.942809    18    0.950000  0.113759   
                 0.999         1.384615  1.120897    13    1.000000  0.000000   

                                  batch_size             
                            count       mean  std count  
optimizer lr     adam_beta2                              
adam      0.0003 0.900         15      128.0  0.0    15  
                 0.990         18      128.0  0.0    18  
                 0.999         13      128.0  0.0    13

## Length costs

- hard to conclude!

In [39]:
fields = ['final_loss', 'final_loss_roles', 'final_length', 'lr', 'adam_beta2', 'random_seed',
          'optimizer', 'temperature', 'length_cost', 'ada_len_cost_thresh']
criterion_1 = (df['mode'] == 'gs')
# criterion_2 = (df['optimizer'] == 'adam')
#criterion_3 = (df['train_test_ratio'] == '-1.0')
a = df.loc[criterion_1][fields].groupby(['ada_len_cost_thresh', 'length_cost'])
print(a)
a.agg(['mean', 'std', 'count'])

final_loss                  final_loss_roles  \
                                      mean        std count             mean   
ada_len_cost_thresh length_cost                                                
0.0                 0.1          20.137739  12.314804     6         7.403351   
                    1.0          32.072582   0.146931     6         7.030446   
5.0                 0.3          11.425177   2.403848     6         7.427288   
                    1.0          15.981870   6.256439     2         8.228920   
10.0                0.1          17.382198   3.228173     6         7.637812   
                    0.3          12.337831   3.742644     8         7.582091   
15.0                0.1          16.128834  10.310162     3         7.894670   
                    0.3          12.044763   2.609016     5         7.588623   
                    1.0          18.941868   6.999175     4         7.917489   

                                                final_length                  \
                                      std count         mean       std count   
ada_len_cost_thresh length_cost                                                
0.0                 0.1          0.672279     6     3.921736  3.277714     6   
                    1.0          0.572143     6     1.000000  0.000000     6   
5.0                 0.3          0.461356     6     7.487077  0.721455     6   
                    1.0          1.158203     2     6.659258  1.896095     2   
10.0                0.1          0.664087     6     8.000000  0.000000     6   
                    0.3          0.554591     8     6.772727  0.933950     8   
15.0                0.1          0.524021     3     8.000000  0.000000     3   
                    0.3          0.475889     5     6.143516  1.158822     5   
                    1.0          0.752509     4     2.970338  0.681872     4   

                                     lr            adam_beta2                  \
                                   mean  std count       mean       std count   
ada_len_cost_thresh length_cost                                                 
0.0                 0.1          0.0003  0.0     6    0.96600  0.051123     6   
                    1.0          0.0003  0.0     6    0.96300  0.048965     6   
5.0                 0.3          0.0003  0.0     6    0.96000  0.046476     6   
                    1.0          0.0003  0.0     2    0.99000  0.000000     2   
10.0                0.1          0.0003  0.0     6    0.96600  0.051123     6   
                    0.3          0.0003  0.0     8    0.94725  0.050627     8   
15.0                0.1          0.0003  0.0     3    0.93300  0.057158     3   
                    0.3          0.0003  0.0     5    0.97200  0.040249     5   
                    1.0          0.0003  0.0     4    0.99000  0.000000     4   

                                random_seed                 temperature  \
                                       mean       std count        mean   
ada_len_cost_thresh length_cost                                           
0.0                 0.1            0.000000  0.000000     6    1.083333   
                    1.0            2.000000  0.894427     6    1.050000   
5.0                 0.3            1.333333  1.032796     6    1.050000   
                    1.0            1.000000  0.000000     2    0.900000   
10.0                0.1            2.666667  0.516398     6    0.966667   
                    0.3            1.500000  1.195229     8    0.950000   
15.0                0.1            2.666667  0.577350     3    0.933333   
                    0.3            2.200000  0.836660     5    0.900000   
                    1.0            1.000000  1.154701     4    0.900000   

                                                 
                                      std count  
ada_len_cost_thresh length_cost                  
0.0                 0.1          0.129099     6  
                    1.0          0.1

# Best runs

In [40]:
pd.set_option('display.max_columns', None)

fields = ['final_length', 'final_loss', 'final_loss_roles',
          'final_length_1.0', 'final_length_2.0', 'final_length_3.0', 
          'ada_len_cost_thresh', 'adam_beta2',
          'length_cost', 'batch_size', 'vocab_size',
          'temperature', 'mode', 'optimizer', 'lr', 'checkpoint_dir', 'receiver_nlayers', 'sender_nlayers',
         'dropout', 'random_seed', 'distance_reg_coef']

criterion_1 = (df['mode'] == 'gs')
criterion_2 = (df['final_loss'] < 10.0)
criterion_3 = (df['final_length'] < 7)

# all runs shld be like that, but I prefer to verify, in case I load the wrong runs
# criterion_6 = (df['receiver_mlp'] == 'True')  
# criterion_7 = (df['sender_squash_output'] >= '0.0')
# criterion_8 = (df['sender_mlp'] == 'True') 
# criterion_9 = (df['batch_size'] == '128') 
criterion_10 = (df['entropy_coef'] == 0.0) 


sub_df = df.loc[criterion_1 & criterion_2 & criterion_3 & criterion_10]
a = sub_df[fields].groupby(['distance_reg_coef', 'ada_len_cost_thresh', 'length_cost',     
                            'lr', 'vocab_size', 'temperature']) 
print(a)
a.agg(['mean', 'std', 'count'])

final_length  \
                                                                                        mean   
distance_reg_coef ada_len_cost_thresh length_cost lr     vocab_size temperature                
0.0               0.0                 0.1         0.0003 64.0       1.25            4.441176   
                  10.0                0.3         0.0003 512.0      0.90            6.648396   
                  15.0                0.3         0.0003 512.0      0.90            6.304813   
1.0               5.0                 0.3         0.0003 512.0      1.25            6.149733   
                  10.0                0.3         0.0003 512.0      1.00            6.822192   

                                                                                           \
                                                                                      std   
distance_reg_coef ada_len_cost_thresh length_cost lr     vocab_size temperature             
0.0               0.0                 0.1         0.0003 64.0       1.25              NaN   
                  10.0                0.3         0.0003 512.0      0.90              NaN   
                  15.0                0.3         0.0003 512.0      0.90              NaN   
1.0               5.0                 0.3         0.0003 512.0      1.25              NaN   
                  10.0                0.3         0.0003 512.0      1.00         0.054356   

                                                                                       \
                                                                                count   
distance_reg_coef ada_len_cost_thresh length_cost lr     vocab_size temperature         
0.0               0.0                 0.1         0.0003 64.0       1.25            1   
                  10.0                0.3         0.0003 512.0      0.90            1   
                  15.0                0.3         0.0003 512.0      0.90            1   
1.0               5.0                 0.3         0.0003 512.0      1.25            1   
                  10.0                0.3         0.0003 512.0      1.00            2   

                                                                                final_loss  \
                                                                                      mean   
distance_reg_coef ada_len_cost_thresh length_cost lr     vocab_size temperature              
0.0               0.0                 0.1         0.0003 64.0       1.25          9.838125   
                  10.0                0.3         0.0003 512.0      0.90          9.761546   
                  15.0                0.3         0.0003 512.0      0.90          8.548177   
1.0               5.0                 0.3         0.0003 512.0      1.25          9.376933   
                  10.0                0.3         0.0003 512.0      1.00          8.726135   

                                                                                           \
                                                                                      std   
distance_reg_coef ada_len_cost_thresh length_cost lr     vocab_size temperature             
0.0               0.0                 0.1         0.0003 64.0       1.25              NaN   
                  10.0                0.3         0.0003 512.0      0.90              NaN   
                  15.0                0.3         0.0003 512.0      0.90              NaN   
1.0               5.0                 0.3         0.0003 512.0      1.25              NaN   
                  10.0                0.3         0.0003 512.0      1.00         0.074429   

                                                                                       \
                                                                                count   
distance_reg_coef ada_len_cost_thresh length_cost lr     vocab_size temperature         
0.0               0.0                 0.1         0.0003 64.0       1.25            1   

## Find a specific run

In [41]:
pd.set_option('display.max_columns', None)

fields = ['final_length', 'final_loss',
          'final_length_1.0', 'final_length_2.0', 'final_length_3.0', 
          'ada_len_cost_thresh', 'adam_beta2',
          'length_cost', 'batch_size',
          'temperature', 'mode', 'optimizer', 'lr', 'checkpoint_dir']

criterion_1 = (df['mode'] == 'gs')
criterion_2 = (df['final_loss'] < 10.0)
criterion_3 = (df['final_length'] < 7)


sub_df = df.loc[criterion_1 & criterion_2 & criterion_3]
print(sub_df)
print(sub_df['checkpoint_dir'].values)

    ada_H_cost_thresh  ada_len_cost_thresh  adam_beta2  batch_size  \
1                 0.0                  0.0       0.900       128.0   
11                0.0                 10.0       0.999       128.0   
27                0.0                 15.0       0.990       128.0   
34                0.0                 10.0       0.900       128.0   
35                0.0                 10.0       0.900       128.0   
40                0.0                  5.0       0.900       128.0   

   checkpoint_best                                     checkpoint_dir  \
1             True  /Tmp/slurm.940288.0/bosctom/proto_3_adam_sd2/6...   
11            True  /Tmp/slurm.944144.0/bosctom/proto_3_adam_sd1/e...   
27            True  /Tmp/slurm.944145.0/bosctom/proto_3_adam_sd1/7...   
34            True  /Tmp/slurm.944144.0/bosctom/proto_3_adam_sd0/1...   
35            True  /Tmp/slurm.944144.0/bosctom/proto_3_adam_sd0/e...   
40            True  /Tmp/slurm.944145.0/bosctom/proto_3_adam_sd2/5...  